# Transfer Learning Model 'Xception' #

In [1]:

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.xception import Xception
from keras import backend as K


Using TensorFlow backend.


In [2]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train+val','train')
     valid_path = os.path.join(base_dir, 'train+val','valid')

     # instantiate data generators
     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96

train_gen, val_gen = get_pcam_generators('C:/Users/Daniel/Documents/')
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
input = Input(input_shape)
# get the pretrained model, cut out the top layer
pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')

layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [3]:
for j in range(14,0,-1): #verander hier het aantal blokken dat je pretrained model heeft
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
    input = Input(input_shape)
    pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')
    for layer in pretrained.layers:
            layer.trainable = False            

    string = 'block'+str(j)
    indices = [i for i, s in enumerate(layernames) if string in s]      
    for layer in pretrained.layers[min(indices):]:
        layer.trainable = True
        
    output = pretrained(input)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(0.5)(output)
    output = Dense(1, activation='sigmoid')(output)

    # Check the trainable status of the individual layers
    #for layer in pretrained.layers:
    #    print(layer, layer.trainable)

    model = Model(input, output)
    model.compile(SGD(lr=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])
    #
    # print a summary of the model on screen
    print(string)
    for i,layer in enumerate(pretrained.layers):
        print(i,layer.name,layer.trainable)
    model.summary()
        
    # save the model and weights
    model_name = 'Xception3 '+string
    model_filepath = model_name + '.json'
    weights_filepath = model_name + '_weights.hdf5'

    model_json = model.to_json() # serialize model to JSON
    with open(model_filepath, 'w') as json_file:
        json_file.write(model_json)


    # define the model checkpoint and Tensorboard callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(os.path.join('logs', model_name))
    callbacks_list = [checkpoint, tensorboard]


    # train the model, note that we define "mini-epochs"
    train_steps = train_gen.n//train_gen.batch_size//20
    val_steps = val_gen.n//val_gen.batch_size//20

    # since the model is trained for only 10 "mini-epochs", i.e. half of the data is
    # not used during training

    history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=10,
                        callbacks=callbacks_list)
    K.clear_session()


block14
0 input_4 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_5 False
13 block2_pool False
14 batch_normalization_5 False
15 add_13 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_6 False
23 block3_pool False
24 batch_normalization_6 False
25 add_14 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_7 False
33 block4_pool False
34 batch_normalization_7 False
35 add_15 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 bloc

Epoch 1/10
225/225 [==============================] - 37s 167ms/step - loss: 0.6007 - acc: 0.6839 - val_loss: 0.8948 - val_acc: 0.6175

Epoch 00001: val_loss improved from inf to 0.89484, saving model to Xception3 block13_weights.hdf5
Epoch 2/10
225/225 [==============================] - 35s 158ms/step - loss: 0.4654 - acc: 0.7840 - val_loss: 0.9853 - val_acc: 0.6225

Epoch 00002: val_loss did not improve from 0.89484
Epoch 3/10
225/225 [==============================] - 35s 158ms/step - loss: 0.4155 - acc: 0.8121 - val_loss: 0.8707 - val_acc: 0.6575

Epoch 00003: val_loss improved from 0.89484 to 0.87067, saving model to Xception3 block13_weights.hdf5
Epoch 4/10
225/225 [==============================] - 35s 158ms/step - loss: 0.4023 - acc: 0.8208 - val_loss: 0.8409 - val_acc: 0.6750

Epoch 00004: val_loss improved from 0.87067 to 0.84092, saving model to Xception3 block13_weights.hdf5
Epoch 5/10
225/225 [==============================] - 35s 158ms/step - loss: 0.3877 - acc: 0.8251 - 

225/225 [==============================] - 41s 182ms/step - loss: 0.3483 - acc: 0.8451 - val_loss: 0.6061 - val_acc: 0.6963

Epoch 00008: val_loss did not improve from 0.56961
Epoch 9/10
225/225 [==============================] - 41s 182ms/step - loss: 0.3531 - acc: 0.8454 - val_loss: 0.5963 - val_acc: 0.7388

Epoch 00009: val_loss did not improve from 0.56961
Epoch 10/10
225/225 [==============================] - 41s 182ms/step - loss: 0.3500 - acc: 0.8435 - val_loss: 0.5415 - val_acc: 0.7188

Epoch 00010: val_loss improved from 0.56961 to 0.54154, saving model to Xception3 block12_weights.hdf5
block11
0 input_2 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_1 False
13 block2_pool False
14 batch_normalization_1 False
15 add_1 False
16 block3_sepcon

Epoch 1/10
225/225 [==============================] - 53s 233ms/step - loss: 0.5780 - acc: 0.6947 - val_loss: 0.6689 - val_acc: 0.5775

Epoch 00001: val_loss improved from inf to 0.66890, saving model to Xception3 block10_weights.hdf5
Epoch 2/10
225/225 [==============================] - 50s 224ms/step - loss: 0.4418 - acc: 0.7989 - val_loss: 0.6009 - val_acc: 0.6637

Epoch 00002: val_loss improved from 0.66890 to 0.60085, saving model to Xception3 block10_weights.hdf5
Epoch 3/10
225/225 [==============================] - 50s 224ms/step - loss: 0.4000 - acc: 0.8193 - val_loss: 0.5671 - val_acc: 0.7238

Epoch 00003: val_loss improved from 0.60085 to 0.56710, saving model to Xception3 block10_weights.hdf5
Epoch 4/10
225/225 [==============================] - 50s 224ms/step - loss: 0.4042 - acc: 0.8189 - val_loss: 0.5703 - val_acc: 0.7275

Epoch 00004: val_loss did not improve from 0.56710
Epoch 5/10
225/225 [==============================] - 50s 224ms/step - loss: 0.3708 - acc: 0.8361 - 

Epoch 8/10
225/225 [==============================] - 55s 245ms/step - loss: 0.3369 - acc: 0.8521 - val_loss: 0.5054 - val_acc: 0.7650

Epoch 00008: val_loss improved from 0.51927 to 0.50543, saving model to Xception3 block9_weights.hdf5
Epoch 9/10
225/225 [==============================] - 55s 245ms/step - loss: 0.3329 - acc: 0.8553 - val_loss: 0.4850 - val_acc: 0.7762

Epoch 00009: val_loss improved from 0.50543 to 0.48500, saving model to Xception3 block9_weights.hdf5
Epoch 10/10
225/225 [==============================] - 55s 245ms/step - loss: 0.3354 - acc: 0.8503 - val_loss: 0.5254 - val_acc: 0.7488

Epoch 00010: val_loss did not improve from 0.48500
block8
0 input_2 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_1 False
13 block2_pool False
14

Epoch 1/10
225/225 [==============================] - 68s 300ms/step - loss: 0.5585 - acc: 0.7149 - val_loss: 0.6291 - val_acc: 0.6225

Epoch 00001: val_loss improved from inf to 0.62913, saving model to Xception3 block7_weights.hdf5
Epoch 2/10
225/225 [==============================] - 65s 288ms/step - loss: 0.4389 - acc: 0.8001 - val_loss: 0.5530 - val_acc: 0.6975

Epoch 00002: val_loss improved from 0.62913 to 0.55300, saving model to Xception3 block7_weights.hdf5
Epoch 3/10
225/225 [==============================] - 65s 288ms/step - loss: 0.4020 - acc: 0.8207 - val_loss: 0.7366 - val_acc: 0.5925

Epoch 00003: val_loss did not improve from 0.55300
Epoch 4/10
225/225 [==============================] - 65s 288ms/step - loss: 0.3849 - acc: 0.8271 - val_loss: 0.5153 - val_acc: 0.7562

Epoch 00004: val_loss improved from 0.55300 to 0.51529, saving model to Xception3 block7_weights.hdf5
Epoch 5/10
225/225 [==============================] - 65s 288ms/step - loss: 0.3600 - acc: 0.8378 - val

225/225 [==============================] - 70s 309ms/step - loss: 0.2877 - acc: 0.8800 - val_loss: 0.7502 - val_acc: 0.6963

Epoch 00010: val_loss did not improve from 0.48667
block5
0 input_2 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_1 False
13 block2_pool False
14 batch_normalization_1 False
15 add_1 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_2 False
23 block3_pool False
24 batch_normalization_2 False
25 add_2 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_3 False
33 block4_poo

Epoch 1/10
225/225 [==============================] - 85s 380ms/step - loss: 0.5595 - acc: 0.7131 - val_loss: 0.6848 - val_acc: 0.6562

Epoch 00001: val_loss improved from inf to 0.68476, saving model to Xception3 block4_weights.hdf5
Epoch 2/10
225/225 [==============================] - 82s 364ms/step - loss: 0.4077 - acc: 0.8178 - val_loss: 0.6427 - val_acc: 0.6663

Epoch 00002: val_loss improved from 0.68476 to 0.64266, saving model to Xception3 block4_weights.hdf5
Epoch 3/10
225/225 [==============================] - 82s 364ms/step - loss: 0.3740 - acc: 0.8319 - val_loss: 0.9520 - val_acc: 0.5900

Epoch 00003: val_loss did not improve from 0.64266
Epoch 4/10
225/225 [==============================] - 82s 364ms/step - loss: 0.3380 - acc: 0.8539 - val_loss: 0.8132 - val_acc: 0.6275

Epoch 00004: val_loss did not improve from 0.64266
Epoch 5/10
225/225 [==============================] - 82s 364ms/step - loss: 0.3388 - acc: 0.8558 - val_loss: 0.7646 - val_acc: 0.6600

Epoch 00005: val_l

block2
0 input_2 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 True
8 block2_sepconv1_bn True
9 block2_sepconv2_act True
10 block2_sepconv2 True
11 block2_sepconv2_bn True
12 conv2d_1 True
13 block2_pool True
14 batch_normalization_1 True
15 add_1 True
16 block3_sepconv1_act True
17 block3_sepconv1 True
18 block3_sepconv1_bn True
19 block3_sepconv2_act True
20 block3_sepconv2 True
21 block3_sepconv2_bn True
22 conv2d_2 True
23 block3_pool True
24 batch_normalization_2 True
25 add_2 True
26 block4_sepconv1_act True
27 block4_sepconv1 True
28 block4_sepconv1_bn True
29 block4_sepconv2_act True
30 block4_sepconv2 True
31 block4_sepconv2_bn True
32 conv2d_3 True
33 block4_pool True
34 batch_normalization_3 True
35 add_3 True
36 block5_sepconv1_act True
37 block5_sepconv1 True
38 block5_sepconv1_bn True
39 block5_sepconv2_act True
40 block5_sepconv2 True
41 block5_sepconv2_b

Epoch 1/10
225/225 [==============================] - 121s 536ms/step - loss: 0.5412 - acc: 0.7301 - val_loss: 0.4176 - val_acc: 0.8125

Epoch 00001: val_loss improved from inf to 0.41764, saving model to Xception3 block1_weights.hdf5
Epoch 2/10
225/225 [==============================] - 115s 513ms/step - loss: 0.3946 - acc: 0.8219 - val_loss: 0.3622 - val_acc: 0.8638

Epoch 00002: val_loss improved from 0.41764 to 0.36225, saving model to Xception3 block1_weights.hdf5
Epoch 3/10
225/225 [==============================] - 115s 512ms/step - loss: 0.3565 - acc: 0.8486 - val_loss: 0.3186 - val_acc: 0.8588

Epoch 00003: val_loss improved from 0.36225 to 0.31861, saving model to Xception3 block1_weights.hdf5
Epoch 4/10
225/225 [==============================] - 115s 511ms/step - loss: 0.3325 - acc: 0.8574 - val_loss: 0.2802 - val_acc: 0.8862

Epoch 00004: val_loss improved from 0.31861 to 0.28023, saving model to Xception3 block1_weights.hdf5
Epoch 5/10
225/225 [============================